Usaremos um modelo GRU para uma tarefa de previsão de série de tempo. O conjunto de dados que usaremos é o conjunto de dados de consumo de energia por hora. O conjunto de dados contém dados de consumo de energia em diferentes regiões dos Estados Unidos registrados de hora em hora.

Estaremos usando a biblioteca PyTorch para implementar o tipo de modelo GRU junto com outras bibliotecas Python comuns usadas na análise de dados.

<h3>Obs: Por motivos de baixos recursos computacionais e tempo, simplifiquei os dados, mas no arquivo data/completo_AEP_hourly.csv você encontra os dados completos<h3/>

In [13]:
import os

import time
from time import process_time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

from tqdm.notebook import tqdm_notebook
from sklearn.preprocessing import MinMaxScaler

# Definindo o diretório raiz de dados
data_dir = "./data/"
print(os.listdir(data_dir))

['pjm_hourly_est.csv', 'AEP_hourly.csv']


In [14]:
pd.read_csv(data_dir+'AEP_hourly.csv').head()

,Datetime,AEP_MW
0,2004-12-31 01:00:00,13478.0
1,2004-12-31 02:00:00,12865.0
2,2004-12-31 03:00:00,12577.0
3,2004-12-31 04:00:00,12517.0
4,2004-12-31 05:00:00,12670.0


Temos um total de 2 arquivos .csv contendo dados de tendência de energia por hora ('pjm_hourly_est.csv' não é usado). Em nossa próxima etapa, leremos esses arquivos e pré-processaremos esses dados nesta ordem:

- Obter os dados de cada etapa de tempo individual e generalizá-los
    - Hour of the day *i.e. 0-23*
    - Day of the week *i.e. 1-7*
    - Month *i.e. 1-12*
    - Day of the year *i.e. 1-365*
    
    
- Dimensionando os dados para valores entre 0 e 1
    - Algoritmos tendem a ter um desempenho melhor ou convergir mais rápido quando os recursos estão em uma escala relativamente semelhante e / ou perto da distribuição normal
     - O dimensionamento preserva a forma da distribuição original e não reduz a importância.
    
    
- Agrupamos os dados em sequências para serem usados como entradas para o modelo e armazenamos seus rótulos correspondentes
   

- As entradas e rótulos serão divididos em conjuntos de treinamento e teste

In [15]:
# Os objetos agrupadores serão armazenados neste dicionário para que nossos dados de teste de saída do modelo possam ser redimensionados durante a avaliação
label_scalers = {}

train_x = []
test_x = {}
test_y = {}

for file in tqdm_notebook(os.listdir(data_dir)): 
    # Ignorando os arquivos que não estamos usando
    if file[-4:] != ".csv" or file == "pjm_hourly_est.csv" or file == "completo_AEP_hourly.csv":
        continue
    
    # Armazene o arquivo csv em um Pandas DataFrame
    df = pd.read_csv(data_dir + file, parse_dates=[0])
    # Processando os dados de tempo em formatos de entrada adequados
    df['hour'] = df.apply(lambda x: x['Datetime'].hour,axis=1)
    df['dayofweek'] = df.apply(lambda x: x['Datetime'].dayofweek,axis=1)
    df['month'] = df.apply(lambda x: x['Datetime'].month,axis=1)
    df['dayofyear'] = df.apply(lambda x: x['Datetime'].dayofyear,axis=1)
    df = df.sort_values("Datetime").drop("Datetime",axis=1)
    
    # Escalonando os dados de entrada
    sc = MinMaxScaler()
    label_sc = MinMaxScaler()
    data = sc.fit_transform(df.values)
    # Obtendo a escala para os rótulos (dados de uso) para que a saída possa ser redimensionada para o valor real durante a avaliação
    label_sc.fit(df.iloc[:,0].values.reshape(-1,1))
    label_scalers[file] = label_sc
    
    # Definindo o período de lookback e divida entradas / rótulos
    lookback = 90
    inputs = np.zeros((len(data)-lookback,lookback,df.shape[1]))
    labels = np.zeros(len(data)-lookback)
    
    for i in range(lookback, len(data)):
        inputs[i-lookback] = data[i-lookback:i]
        labels[i-lookback] = data[i,0]
    inputs = inputs.reshape(-1,lookback,df.shape[1])
    labels = labels.reshape(-1,1)
    
    # Dividindo os dados em porções de treinamento / teste e combinando todos os dados
    test_portion = int(0.1*len(inputs))
    if len(train_x) == 0:
        train_x = inputs[:-test_portion]
        train_y = labels[:-test_portion]
    else:
        train_x = np.concatenate((train_x,inputs[:-test_portion]))
        train_y = np.concatenate((train_y,labels[:-test_portion]))
    test_x[file] = (inputs[-test_portion:])
    test_y[file] = (labels[-test_portion:])

  0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
# print(train_x.shape)

Para melhorar a velocidade de nosso treinamento, podemos processar os dados em lotes para que o modelo não precise atualizar seus pesos com tanta frequência. As classes Torch * Dataset * e * DataLoader * são úteis para dividir nossos dados em lotes e embaralhá-los.

In [17]:
batch_size = 1024

train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, drop_last=True)

In [18]:
# is_cuda = torch.cuda.is_available()
# if is_cuda:
#     device = torch.device("cuda")
# else:
#     device = torch.device("cpu")
device = torch.device("cpu")

A seguir, definiremos a estrutura do modelo GRU. 

In [19]:
class GRUNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, drop_prob=0.2):
        super(GRUNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        
    def forward(self, x, h):
        out, h = self.gru(x, h)
        out = self.fc(self.relu(out[:,-1]))
        return out, h
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device)
        return hidden

In [20]:
def train(train_loader, learn_rate, hidden_dim=256, EPOCHS=5, model_type="GRU"):
    
    input_dim = next(iter(train_loader))[0].shape[2]
    output_dim = 1
    n_layers = 2
    # Instanciando o modelo
    model = GRUNet(input_dim, hidden_dim, output_dim, n_layers)
    
    # Definição de função de perda e otimizador
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learn_rate)
    
    model.train()
    print("Starting Training of {} model".format(model_type))
    epoch_times = []
    # Iniciando loop de treinamento
    for epoch in range(1,EPOCHS+1):
        start_time = process_time()
        h = model.init_hidden(batch_size)
        avg_loss = 0.
        counter = 0
        for x, label in train_loader:
            counter += 1
            h = h.data
            
            out, h = model(x.to(device).float(), h)
            loss = criterion(out, label.to(device).float())
            loss.backward()
            optimizer.step()
            avg_loss += loss.item()
            if counter%200 == 0:
                print("Epoch {}......Step: {}/{}....... Average Loss for Epoch: {}".format(epoch, counter, len(train_loader), avg_loss/counter))
        current_time = time.process_time()
        print("Epoch {}/{} Done, Total Loss: {}".format(epoch, EPOCHS, avg_loss/len(train_loader)))
        print("Time Elapsed for Epoch: {} seconds".format(str(current_time-start_time)))
        epoch_times.append(current_time-start_time)
    print("Total Training Time: {} seconds".format(str(sum(epoch_times))))
    return model

def evaluate(model, test_x, test_y, label_scalers):
    model.eval()
    outputs = []
    targets = []
    start_time = process_time()
    for i in test_x.keys():
        inp = torch.from_numpy(np.array(test_x[i]))
        labs = torch.from_numpy(np.array(test_y[i]))
        h = model.init_hidden(inp.shape[0])
        out, h = model(inp.to(device).float(), h)
        outputs.append(label_scalers[i].inverse_transform(out.cpu().detach().numpy()).reshape(-1))
        targets.append(label_scalers[i].inverse_transform(labs.numpy()).reshape(-1))
    print("Evaluation Time: {}".format(str(time.process_time()-start_time)))
    sMAPE = 0
    for i in range(len(outputs)):
        sMAPE += np.mean(abs(outputs[i]-targets[i])/(targets[i]+outputs[i])/2)/len(outputs)
    print("sMAPE: {}%".format(sMAPE*100))
    return outputs, targets, sMAPE

In [21]:
lr = 0.001
gru_model = train(train_loader, lr, model_type="GRU")

Starting Training of GRU model
Epoch 1/5 Done, Total Loss: 0.1798754185438156
Time Elapsed for Epoch: 37.031527326 seconds
Epoch 2/5 Done, Total Loss: 0.0985664427280426
Time Elapsed for Epoch: 43.849040951999996 seconds
Epoch 3/5 Done, Total Loss: 0.04450027644634247
Time Elapsed for Epoch: 43.026285185999996 seconds
Epoch 4/5 Done, Total Loss: 0.026159686967730522
Time Elapsed for Epoch: 53.944850213999985 seconds
Epoch 5/5 Done, Total Loss: 0.06714832037687302
Time Elapsed for Epoch: 39.53589641100007 seconds
Total Training Time: 217.38760008900005 seconds


In [22]:
gru_outputs, targets, gru_sMAPE = evaluate(gru_model, test_x, test_y, label_scalers)

Evaluation Time: 1.8103908580000052
sMAPE: 4.945344692085785%
